# FIT5196-S1-2019 Assessment 1
## Task 2
### Student Name: Mranali Mehta
### Student ID: 29778271
Date: 14/04/2018
Environment: Python 3.6.0 and Anaconda 4.3.0 (64-bit)

##### Libraries used :
re : for regular expression for tokenizing the text.

nltk.collocations : used for creating bigrams

itertools : for iterating over the dictionary

MWETokenizer : To tokenise the bigrams 

PorterStemmer : To stem the token




## 1. Import the libraries required for performing the task.

In [1]:
import re
from nltk.collocations import *
from itertools import chain
import nltk
from nltk.tokenize import MWETokenizer
from nltk.stem import PorterStemmer
from __future__ import division
from nltk.probability import *



## 2. Open the files required. 
###### Initialize the data structure to be used, regular exoression to be used.
###### Open the input files to read.

In [2]:
input_file = open("29778271.txt",'r')

text_in_file = input_file.readlines()

unit_info_dict = {}

unit_regex = r"[A-Z]{3}[0-9]{4}\n"


## 3. Clean the data after reading it from the text file & Normalise the first letter of sentence. 
###### Cleaning is performed as:
-> Removing the '\n' from the end of tokens.

-> Removing the special characters or brackets from the tokens.

-> Replace the unit code occuring between the text which can create ambiguity.

-> Normalise the first letter in the token at the start of each sentence, leaving the upper case token coming inside the text.

-> Also remove the alphanumeric tokens.

Finally store the list of cleaned and normalised tokens in the form of list into a dictionary whose key is unit_code.

In [3]:

## Cleaning the unit info in the text file for further pre-processing
## Normalising the first letter in each sentence.

for index,line in enumerate(text_in_file):
    unit_info_ls = []
    count = 0
    if re.search(unit_regex,line):
        unit_id = re.search(unit_regex,line).group()
        if unit_id[-1] == '\n':
            unit_id = unit_id.replace('\n','')
        i = index + 1
        token = text_in_file[i]
        while count < 3:
            if token == '\n':
                count += 1
            else:
                if token[0] == "[":
                    token = token.replace("[","")
                    if token[0] == "\'":
                        token = token.replace("\'","")
                if token[-1] == "\n":
                    token = token.replace("\n","")
                if token[-1] == "]":
                    token = token.replace("]","")
                if re.search(r"[A-Z]{3}[0-9]{4}",token):
                    token = token.replace(re.search(r"[A-Z]{3}[0-9]{4}",token).group()," ")
                if token[0].isupper():
                    token = token.replace(token[0],token[0].lower())
                if not token.isalnum():
                    unit_info_ls.append(token)
            i += 1
            token = text_in_file[i]
        unit_info_dict[unit_id] = unit_info_ls




## 4. Tokenising the cleaned text.

-> Make use of the regex given and tokenize the text based on that. 

-> Tokenisation is done unit wise. Information of one unit is tokenized at a time and stored in one list.

-> Then this list is added to the dictionary with key as its unit ID.



In [4]:
# tokenizing the text
reg = r"\w+(?:[-']\w+)?"
for key in unit_info_dict.keys():
    ui =[]
    local_str = ""
    for each_line in unit_info_dict[key]:
        local_str = local_str + " " + each_line 
    ui = re.findall(reg,local_str)
    unit_info_dict[key] = ui
    

## 5. Removing the stop words from the tokens list.

Read the stopwords provided in the text file. Compare them and remove them.

In [5]:
# reading the stop words and removing the \n from each stop word.        
stopwords = open("stopwords_en.txt",'r')
stopwords_list = stopwords.readlines()
stop_words = []
for stopword in stopwords_list:
    if stopword[-1] == '\n':
        stopword = stopword.replace("\n","")
        stop_words.append(stopword)

# removing stop words
for key,value in unit_info_dict.items():
    filtered_unit_info = []
    for token in value:
        if token not in stop_words:
            filtered_unit_info.append(token)
    unit_info_dict[key] = filtered_unit_info



## 6. Remove tokens with length less than 3

Look for tokens with length less than 3 and remove them.

Also, I removed 'the' and 'this' separately as they were not removed from the tokens and were occuring so many times.

In [14]:
# removing the tokens from the unit_info_dict which is less than 3 in length
for key,value in unit_info_dict.items():
    for token in value:
        if len(token) < 3:
            value.remove(token)
            
# Removing the, this from the unit_info
for key,value in unit_info_dict.items():
    for token in value:
        if token in ['the','this','The','This']:
            value.remove(token)
        


## 7. Find the biagrams, stem the tokens and then tokenize the biagrams.

First find out the top 200 bigrams.

Tokenise them nd then stem the tokens.

In [15]:
# Finding the bigrams 
all_tokens = list(chain.from_iterable(unit_info_dict.values()))
bigram_measures = nltk.collocations.BigramAssocMeasures()
bigram_finder = nltk.collocations.BigramCollocationFinder.from_words(all_tokens)
bigram_finder.apply_freq_filter(10)
bigram_finder.apply_word_filter(lambda w: len(w) < 3)
all_bigrams = bigram_finder.nbest(bigram_measures.pmi, 200)

# re-tokenising the bigrams after stemming the tokens
unit_info_bigram = {}
mwetokenizer = MWETokenizer(all_bigrams)
for key,value in unit_info_dict.items():
    value_colloc = mwetokenizer.tokenize(value)
    unit_info_bigram[key] = value_colloc
    
unit_info_dict = unit_info_bigram
    
# Stemming the unit_info
stemmer = PorterStemmer()
for key,value in unit_info_dict.items():
    stem_ls = []
    for w in value:
        if w[0].isupper():
            stem_ls.append(w)
        else:
            stem_ls.append(stemmer.stem(w))
    unit_info_dict[key] = stem_ls


## 8. Creating the dictionary

Iterate through the list of values in the dictionary and form a list and then pass it to set to generate unique list of words called as dictionary.

Remove the words which occur in more than 95% of the units or less than 5% of the units.

Finally, store it in alphabetical order.

In [16]:
# Creating the vocab-dictionary
words = list(chain.from_iterable(unit_info_dict.values()))
vocab = set(words)

# Removing words with frequency more than 95% and less than 5% from the dictionary
less_than_5 = []
more_than_95 = []
vocab_ls = []

for each in vocab:
    word_count = 0
    for value in unit_info_dict.values():
        if each in value:
            word_count += 1
    if word_count < 10:
        less_than_5.append(each)
    elif word_count > 190:
        more_than_95.append(each)
    else:
        vocab_ls.append(each)

# After removing rare tokens and most frequent tokens from the vocab we can finally add all the 200 bigrams to the vocab
bigrams_ls = list(chain.from_iterable(unit_info_bigram.values()))
bigrams = set(bigrams_ls)
vocab_ls.sort()
vocab = set(vocab_ls)



## 9. Write the words to the vocab.txt with indexing.
## Add the top 200 bigrams in the vocab.

Finally write the vocabulary generated to the text file with proper indexing.

Then add the 200 bigrams to the vocab.txt if they do not exist in vocab so as to avoid the duplication of the vocab.

In [17]:
# writing to vocab.txt
vocab_output = open("29778271_vocab.txt",'w')
index = 0
for unique in vocab_ls:
    vocab_output.write(unique + ":" + str(index) + "\n")
    index += 1
for bigram in bigrams:
    if "_" in bigram:
        if bigram not in vocab_ls:
            vocab_output.write(bigram + ":" + str(index) + "\n")
            index += 1
            
vocab_output.close()



## 10. Generate the countVec.

The last step to generate the space vector model.

Read the words, find the frequency distribution of each word per unit and write the corresponding index for that word with its frequency into a text file.

Repeat the same thing for all the units.

In [18]:
# CountVec Generation:

count_vec_output = open("29778271_countVec.txt",'w')

vocab_dict = {}
i = 0
for w in vocab_ls:
    vocab_dict[w] = i
    i = i + 1

for key,val in unit_info_dict.items():
    count_vec_output.write(key + ',')
    word_index = []
    for word in val:
        if word in vocab_dict.keys():
            word_index.append(vocab_dict[word])
    for k, v in FreqDist(word_index).items():
        count_vec_output.write("{}:{} ".format(k,v) + ',')
    count_vec_output.write('\n')
count_vec_output.close()

# finally close the input file as well.
input_file.close()